In [109]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anime-txt-5000/anime_article_5000.txt
/kaggle/input/titanic-toy/titanic_toy.csv
/kaggle/input/anime-manga-split-random/anime_manga_split.pdf
/kaggle/input/sample-table/sample_table.pdf


In [64]:
# pip install langchain
# pip install langchain_openai
# pip install langchain_community
# pip install pdfplumber

In [65]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain.schema.runnable import RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Langchain Documentation

https://python.langchain.com/docs/integrations/document_loaders/

# 1. Text File Loader

In [66]:
loader = TextLoader('/kaggle/input/anime-txt-5000/anime_article_5000.txt', encoding='utf-8')

docs = loader.load()

print(type(docs))

# print(docs[0].metadata)


# print(docs[0].page_content)

<class 'list'>


In [115]:
api_key='your_api_key'
model = ChatOpenAI(api_key=api_key)

In [68]:
prompt = PromptTemplate(
    template='Write a short summary on the following article: {article}',
    input_variables=['article']
)

In [69]:
str_parser = StrOutputParser()

In [70]:
chain = prompt | model | str_parser


result = chain.invoke({'article': docs[0].page_content })

print(result)

The article "Anime: The Expanding Universe of Japanese Animation" delves into the rich history, evolution, genres, global impact, economic significance, and future prospects of anime. From its humble beginnings in the early 20th century to its current status as a multi-billion-dollar industry, anime has evolved into a powerful cultural connector that resonates with audiences of all ages worldwide. The article highlights key milestones in anime's history, explores its diverse range of genres, discusses its cultural impact, and sheds light on the economic aspects of the anime industry. With advancements in technology and increasing global collaborations, anime's future looks promising as it continues to inspire creativity and storytelling across the globe. Ultimately, anime is not just entertainment but a significant art form that will continue to shape and influence generations to come.


# 2. PDF Loader

e.g. **PyPDFLoader** used to simple text based pdfs

**Other PDF Loaders are**
1. For text based pdfs: PyPDFLoader
2. tables/columns: PDFPlumberLoader
3. Images: UnstructuredPDFLoader / AmazonTextractPDFLoader / PyMuPDFLoader
   
   

In [71]:
from langchain_community.document_loaders import PyPDFLoader, PDFPlumberLoader

In [72]:
py_pdf_loader = PyPDFLoader('/kaggle/input/anime-manga-split-random/anime_manga_split.pdf')

In [73]:
pdf_docs = py_pdf_loader.load()

In [74]:
# print content and meta data of each page
#for page in pdf_docs:
#    print(f"page_ content: {page.page_content}")
#    print("\n")
#    for items in page.metadata.items():
#        print(items)
#    print("\n\n\n")

print(len(pdf_docs)) # 3

3


In [75]:
plumber_pdf_loader = PDFPlumberLoader('/kaggle/input/sample-table/sample_table.pdf')

plumber_docs = plumber_pdf_loader.load()

In [76]:
print(len(plumber_docs)) # 3

3


In [77]:
# for page in plumber_docs:
#    print(f"page_ content: {page.page_content}")
#    print("\n")
#    for items in page.metadata.items():
#        print(items)
#    print("\n\n\n")

Output

page_ content: ID Name Age Country Score
1 Hannah 58 Australia 85
2 Julia 29 Brazil 54
3 Julia 41 UK 91
...



('source', '/kaggle/input/sample-table/sample_table.pdf')
('file_path', '/kaggle/input/sample-table/sample_table.pdf')
('page', 0)
('total_pages', 3)
('Author', '(anonymous)')
('CreationDate', "D:20250908085348+00'00'")
('Creator', '(unspecified)')
('Keywords', '')
('ModDate', "D:20250908085348+00'00'")
('Producer', 'ReportLab PDF Library - www.reportlab.com')
('Subject', '(unspecified)')
('Title', '(anonymous)')
('Trapped', 'False')




page_ content: ID Name Age Country Score
37 Charlie 40 India 58
38 Hannah 21 Germany 82
...



('source', '/kaggle/input/sample-table/sample_table.pdf')
('file_path', '/kaggle/input/sample-table/sample_table.pdf')
('page', 1)
('total_pages', 3)
('Author', '(anonymous)')
('CreationDate', "D:20250908085348+00'00'")
('Creator', '(unspecified)')
('Keywords', '')
('ModDate', "D:20250908085348+00'00'")
('Producer', 'ReportLab PDF Library - www.reportlab.com')
('Subject', '(unspecified)')
('Title', '(anonymous)')
('Trapped', 'False')




page_ content: ID Name Age Country Score
73 Alice 20 UK 64
100 David 22 France 72
...

('source', '/kaggle/input/sample-table/sample_table.pdf')
('file_path', '/kaggle/input/sample-table/sample_table.pdf')
('page', 2)
('total_pages', 3)
('Author', '(anonymous)')
('CreationDate', "D:20250908085348+00'00'")
('Creator', '(unspecified)')
('Keywords', '')
('ModDate', "D:20250908085348+00'00'")
('Producer', 'ReportLab PDF Library - www.reportlab.com')
('Subject', '(unspecified)')
('Title', '(anonymous)')
('Trapped', 'False')

# 3. Directory Loader

Loads all/specific similar type files from a directory 

# Load vs LazyLoad

# Load
Use .load() to synchronously load into memory all Documents,

# LazyLoad
If we're loading a large number of Documents and our downstream operations can be done over subsets of all loaded Documents, we can lazily load our Documents one at a time to minimize our memory footprint

In [78]:
from langchain_community.document_loaders import DirectoryLoader

In [93]:
# load all pdf file from the directory
# /kaggle/input/anime-txt-5000/anime_article_5000.txt
# /kaggle/input/anime-manga-split-random/anime_manga_split.pdf
# /kaggle/input/sample-table/sample_table.pdf


directory_loader = DirectoryLoader(
    path = '/kaggle/input',
    glob = '**/*.pdf', # loads all the pdf documents from the path
    loader_cls = PyPDFLoader,
    show_progress=True
)

#directory_pdf_docs = directory_loader.load()
directory_pdf_docs = directory_loader.lazy_load()

In [94]:
#print(len(directory_pdf_docs)) # 6 - 3 pages in each pdf

print(directory_pdf_docs)

for  doc in directory_pdf_docs:
    print(doc.metadata)

<generator object DirectoryLoader.lazy_load at 0x78021b1395d0>


100%|██████████| 2/2 [00:00<00:00, 40.43it/s]

{'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-08T08:41:58+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-08T08:41:58+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '/kaggle/input/anime-manga-split-random/anime_manga_split.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}
{'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-08T08:41:58+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-08T08:41:58+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '/kaggle/input/anime-manga-split-random/anime_manga_split.pdf', 'total_pages': 3, 'page': 1, 'page_label': '2'}
{'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-08T08:41:58+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '20

In [96]:
# contains content and metdata of each page in the pdf


# for page in directory_pdf_docs:
#    print(f"page_ content: {page.page_content}")
#    print("\n")
#    for items in page.metadata.items():
#        print(items)
#    print("\n\n\n")

#print(directory_pdf_docs[2].metadata)

output

{'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-08T08:41:58+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-08T08:41:58+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': '/kaggle/input/anime-manga-split-random/anime_manga_split.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3'}

# 4. WebBased Loader

In [98]:
from langchain_community.document_loaders import WebBaseLoader

In [107]:
url = ['https://arxiv.org/', 'https://python.langchain.com/docs/integrations/document_loaders/arxiv/#lazy-load']

web_loader = WebBaseLoader(url)

web_docs = web_loader.load()
web_docs_lazy = web_loader.lazy_load()

In [106]:
# print(web_docs[0].page_content)
print(web_docs[0].metadata)
print(len(web_docs))

# output

# {'source': 'https://arxiv.org/', 'title': 'arXiv.org e-Print archive', 'language': 'en'}
# 1

{'source': 'https://arxiv.org/', 'title': 'arXiv.org e-Print archive', 'language': 'en'}
2


In [108]:
for doc in web_docs_lazy:
    print(doc.metadata)

    
# output
# {'source': 'https://arxiv.org/', 'title': 'arXiv.org e-Print archive', 'language': 'en'}
# {'source': 'https://python.langchain.com/docs/integrations/document_loaders/arxiv/#lazy-load', 'title': 'ArxivLoader | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'arXiv is an open-access archive for 2 million scholarly articles in the fields of physics, mathematics, computer science, quantitative biology, quantitative finance, statistics, electrical engineering and systems science, and economics.', 'language': 'en'}

{'source': 'https://arxiv.org/', 'title': 'arXiv.org e-Print archive', 'language': 'en'}
{'source': 'https://python.langchain.com/docs/integrations/document_loaders/arxiv/#lazy-load', 'title': 'ArxivLoader | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'description': 'arXiv is an open-access archive for 2 million scholarly articles in the fields of physics, mathematics, computer science, quantitative biology, quantitative finance, statistics, electrical engineering and systems science, and economics.', 'language': 'en'}


# 5. CSV Loader


In [110]:
from langchain_community.document_loaders import CSVLoader

In [111]:
csv_loader = CSVLoader('/kaggle/input/titanic-toy/titanic_toy.csv')

csv_doc = csv_loader.load()

In [114]:
# print(len(csv_doc))

# for doc in csv_doc:
#     print(doc.metadata)

print(csv_doc[0].page_content)


Age: 22.0
Fare: 7.25
Family: 1
Survived: 0


#output 

891
{'source': '/kaggle/input/titanic-toy/titanic_toy.csv', 'row': 0}

{'source': '/kaggle/input/titanic-toy/titanic_toy.csv', 'row': 1}

{'source': '/kaggle/input/titanic-toy/titanic_toy.csv', 'row': 2}

{'source': '/kaggle/input/titanic-toy/titanic_toy.csv', 'row': 3}

{'source': '/kaggle/input/titanic-toy/titanic_toy.csv', 'row': 4}

...



print(csv_doc[0].page_content) :

Age: 22.0
Fare: 7.25
Family: 1
Survived: 0